<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Advanced/ProtT5-XL-UniRef50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Important Notes:
1. ProtT5-XL-UniRef50 has both encoder and decoder, for feature extraction we only load and use the encoder part.
2. Loading only the encoder part, reduces the inference speed and the GPU memory requirements by half.
2. In order to use ProtT5-XL-UniRef50 encoder, you must install the latest huggingface transformers version from their GitHub repo.
3. If you are intersted in both the encoder and decoder, you should use T5Model rather than T5EncoderModel.

<h3>Extracting protein sequences' features using ProtT5-XL-UniRef50 pretrained-model</h3>

**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q SentencePiece transformers

     |████████████████████████████████| 1.2MB 14.6MB/s 
     |████████████████████████████████| 2.1MB 57.3MB/s 
     |████████████████████████████████| 890kB 68.2MB/s 
     |████████████████████████████████| 3.2MB 52.9MB/s 


In [2]:
import torch
from transformers import T5EncoderModel, T5Tokenizer
import re
import numpy as np
import gc

<b>2. Load the vocabulary and ProtT5-XL-UniRef50 Model<b>

In [3]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )

In [4]:
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

Some weights of the model checkpoint at Rostlab/prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.embed_tokens.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.SelfAtte

In [5]:
gc.collect()

1092

<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
model = model.to(device)
model = model.eval()

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [8]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [9]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [10]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True)

In [11]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

<b>6. Extracting sequences' features and load it into the CPU if needed<b>

In [12]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)

In [13]:
embedding = embedding.last_hidden_state.cpu().numpy()

<b>7. Remove padding (\<pad\>) and special tokens (\</s\>) that is added by ProtT5-XL-UniRef50 model<b>

In [14]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][:seq_len-1]
    features.append(seq_emd)

In [15]:
print(features)

[array([[ 0.22006361, -0.10348098, -0.24565521, ...,  0.07009563,
         0.18895069, -0.12561195],
       [ 0.1083971 , -0.14164554, -0.26113322, ...,  0.07524937,
        -0.19057553,  0.14047636],
       [ 0.20587245, -0.06389292, -0.34735742, ..., -0.02512391,
         0.00397155,  0.00148442],
       ...,
       [ 0.40880167, -0.08833071, -0.1241793 , ...,  0.07904121,
         0.47919267,  0.16784588],
       [ 0.09277282, -0.08904045, -0.2286644 , ..., -0.12425309,
         0.15413281, -0.22367832],
       [ 0.3757485 , -0.05730624, -0.1993564 , ...,  0.04669633,
         0.27533916, -0.10681015]], dtype=float32), array([[ 0.18192561, -0.10257894, -0.25567392, ..., -0.13439739,
        -0.13265462,  0.06053399],
       [ 0.14549373, -0.15877174, -0.03994372, ..., -0.0298596 ,
        -0.3224525 ,  0.07564054],
       [ 0.27877027, -0.09199824, -0.14064308, ..., -0.08892573,
         0.10121177, -0.15753417],
       [ 0.29990828, -0.24917187, -0.26971292, ..., -0.02662831,
     